In [1]:
import xarray as xr
import pandas as pd
import os
import glob
import numpy as np

In [2]:
lat_min = -6.67 # Selatan (Lebih negatif)
lat_max = -6.25 # Utara (Kurang negatif)
lon_min = 107.87
lon_max = 108.60

lat_batas_indramayu = slice(lat_max, lat_min) # Utara ke Selatan
lon_batas_indramayu = slice(lon_min, lon_max)

In [8]:
data_folder = 'D:\\aaUsers\\Downloads\\ESACCI\\' 

# Cari semua file NetCDF (*.nc) secara rekursif di folder utama dan subfoldernya (**)
list_of_local_files = glob.glob(os.path.join(data_folder, '**', '*.nc'), recursive=True)

if not list_of_local_files:
    raise FileNotFoundError(f"❌ ERROR: Tidak ditemukan file .nc di {data_folder}. Pastikan semua file ada di folder tersebut atau di subfoldernya.")

print(f"✅ Ditemukan total {len(list_of_local_files)} file harian untuk digabungkan.")

✅ Ditemukan total 259 file harian untuk digabungkan.


In [9]:
# Fungsi Preprocessing (Potong wilayah Indramayu)
def preprocess_sm(ds):
    # Hanya potong variabel 'sm' dan dimensinya. Kita tahu 'sm' ada.
    ds_cropped = ds['sm'].sel(lat=lat_batas_indramayu, lon=lon_batas_indramayu)
    return ds_cropped

print("Memuat dan memotong semua file menjadi satu dataset. Ini mungkin perlu waktu...")
# Muat semua file ke dalam satu Dataset
ds_indramayu_all = xr.open_mfdataset(
    list_of_local_files,
    combine='by_coords',
    join='override',
    parallel=True,
    engine='netcdf4',
    preprocess=preprocess_sm # Proses pemotongan dijalankan saat memuat
)

print("Dataset gabungan berhasil dimuat dan dipotong.")

Memuat dan memotong semua file menjadi satu dataset. Ini mungkin perlu waktu...
Dataset gabungan berhasil dimuat dan dipotong.


In [14]:
sm_harian_mean = ds_indramayu_all.mean(['lat', 'lon'])

# 2. Resampling ke rata-rata Mingguan (W-SUN: berakhir hari Minggu)
# MENDESAIN sm_mingguan_mean
sm_mingguan_mean = sm_harian_mean.resample(time="W-SUN").mean()

In [15]:
sm_df_mingguan = sm_mingguan_mean.to_pandas()

# 2. Mengganti nama kolom secara langsung menggunakan .rename()
# Karena .to_pandas() menghasilkan DataFrame 1 kolom, kita ganti nama kolom pertama ([0]).
sm_df_mingguan = sm_df_mingguan.rename(columns={sm_df_mingguan.columns[0]: 'SM_Rata2_Mingguan'})
sm_df_mingguan.index.name = 'Tanggal'


print("\n--- DATA RATA-RATA MINGGUAN INDRAMAYU (2014-2023) ---")
print(f"Total titik data: {len(sm_df_mingguan)}")
print(sm_df_mingguan.head())
print(sm_df_mingguan.tail())


--- DATA RATA-RATA MINGGUAN INDRAMAYU (2014-2023) ---
Total titik data: 261
            SM_Rata2_Mingguan
Tanggal                      
2014-01-05           0.396496
2014-01-12           0.370946
2014-01-19           0.395690
2014-01-26           0.366466
2014-02-02           0.379744
            SM_Rata2_Mingguan
Tanggal                      
2018-12-02           0.354329
2018-12-09           0.367088
2018-12-16           0.398260
2018-12-23           0.352794
2018-12-30           0.383978


In [16]:
output_filename = 'SM_Rata2_Mingguan_Indramayu_2014_2023.xlsx'

print("\n--- Menyimpan Data ke File Excel (.xlsx) ---")

try:
    # Menggunakan to_excel untuk menyimpan DataFrame ke format XLSX
    # index=True karena Anda ingin kolom 'Tanggal' (index) ikut tersimpan
    sm_df_mingguan.to_excel(output_filename, index=True)
    
    # Cek lokasi file
    full_path = os.path.abspath(output_filename)
    
    print(f"✅ Data berhasil disimpan ke: {output_filename}")
    print(f"File dapat ditemukan di: {full_path}")
    
except Exception as e:
    print(f"❌ GAGAL menyimpan ke Excel. Pastikan library 'openpyxl' terinstal.")
    print(f"Error: {e}")


--- Menyimpan Data ke File Excel (.xlsx) ---
✅ Data berhasil disimpan ke: SM_Rata2_Mingguan_Indramayu_2014_2023.xlsx
File dapat ditemukan di: d:\Github\MPDW\Pertemuan 9\SM_Rata2_Mingguan_Indramayu_2014_2023.xlsx
